# Project Scoping

## 1. Project Goals
### Exploratory Analysis

- Analyze demographic distributions and patterns (age, location, education, etc.)
- Examine correlations between user attributes and preferences
- Visualize key trends that might guide further analysis
- Identify data quality issues early (missing values, outliers)

### Predictive Modeling

- Predict undisclosed attributes based on essay content
- Build classification models for relationship status or preferences
- Create models to predict compatibility between users

### Text Analysis

- Apply basic NLP to identify common topics across profiles
- Compare language use across different demographic groups
- Extract interests/hobbies from essays to create structured features
[...]

### Recommendation System
- Use similarity metrics to recommend potential matches
- Incorporate text-based features from your NLP analysis
- Create a simple content-based recommendation algorithm

2. Data Assessment
On profiles.csv file, we have:

- Demographic information (age, body type, ethnicity, etc.)
- Multiple essay responses (self-descriptions, interests, etc.)
- Location data
- Educational background
- Dating preferences

In [1]:
import pandas as pd

df = pd.read_csv('profiles.csv')
df.head()

,age,body_type,diet,drinks,drugs,education,essay0,essay1,essay2,essay3,...,location,offspring,orientation,pets,religion,sex,sign,smokes,speaks,status
0,22,a little extra,strictly anything,socially,never,working on college/university,about me:<br />\n<br />\ni would love to think...,currently working as an international agent fo...,making people laugh.<br />\nranting about a go...,"the way i look. i am a six foot half asian, ha...",...,"south san francisco, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism and very serious about it,m,gemini,sometimes,english,single
1,35,average,mostly other,often,sometimes,working on space camp,i am a chef: this is what that means.<br />\n1...,dedicating everyday to being an unbelievable b...,being silly. having ridiculous amonts of fun w...,NaN,...,"oakland, california","doesn&rsquo;t have kids, but might want them",straight,likes dogs and likes cats,agnosticism but not too serious about it,m,cancer,no,"english (fluently), spanish (poorly), french (...",single
2,38,thin,anything,socially,NaN,graduated from masters program,"i'm not ashamed of much, but writing public te...","i make nerdy software for musicians, artists, ...",improvising in different contexts. alternating...,my large jaw and large glasses are the physica...,...,"san francisco, california",NaN,straight,has cats,NaN,m,pisces but it doesn&rsquo;t matter,no,"english, french, c++",available
3,23,thin,vegetarian,socially,NaN,working on college/university,i work in a library and go to school. . .,reading things written by old dead people,playing synthesizers and organizing books acco...,socially awkward but i do my best,...,"berkeley, california",doesn&rsquo;t want kids,straight,likes cats,NaN,m,pisces,no,"english, german (poorly)",single
4,29,athletic,NaN,socially,never,graduated from college/university,hey how's it going? currently vague on the pro...,work work work work + play,creating imagery to look at:<br />\nhttp://bag...,i smile a lot and my inquisitive nature,...,"san francisco, california",NaN,straight,likes dogs and likes cats,NaN,m,aquarius,no,english,single
